In [2]:
def getWhatsappTXT():
    try: 
        file=open("צאט WhatsApp עם יום הולדת בנות לנויה.txt","r", encoding="UTF-8")
        filePerline=file.readlines()
        return filePerline
        
    except: 
        print('file can not be open')
        quite()

In [3]:
def NumOrNameToID(file):
    participant=dict()
    idcount=1
    for line in file:
        line=line.rstrip()
        if (line.count(":")<2 or line.find(":",line.find("-"))==-1):
            continue
        startName_num=line.find("-")
        endName_num=line.find(":",startName_num)
        numOrname=line[startName_num+1:endName_num-1]
        if numOrname not in participant:
            participant[numOrname]=idcount
            idcount=idcount+1
    return participant

In [12]:
def DataOfMessages(file, participant):
    messages=dict()
    fullmessages=[]
    countline=0
    for line in file:
        line=line.rstrip()
        countline=countline+1
        if (line.count(":")<2 or line.find(":",line.find("-"))==-1):
            continue
        enddate=line.find("-")
        startname_num=line.find("-")
        endname_num=line.find(":",startname_num)
        DateAndHour=line[:enddate-1]
        messages.update({"datetime" : DateAndHour})
        thename_num=line[startname_num+1:endname_num-1]
        Theid=participant[thename_num]
        messages.update({"id" : Theid})
        if countline<len(file):
            nextrow=file[countline]
            if (nextrow.find(":")==-1):
                Text=line[endname_num+1:]+ nextrow.rstrip()
            else:
                Text=line[endname_num+1:]
        else:
            Text=line[endname_num+1:]
        messages.update({"text" : Text})
        messages_copy = messages.copy()
        fullmessages.append(messages_copy)
    return fullmessages

In [7]:
def DataOfGroup(file, participant):
    Groupdata=dict()
    import re
    for line in file:
        correct_line=line.rstrip()
        if (correct_line.count(":")<2 or correct_line.find(":",correct_line.find("-"))==-1):
            if re.search("נוצרה על ידי",correct_line):
                BeforeTitle=correct_line.find("הקבוצה")
                AfterTitle=correct_line.find("נוצרה על ידי",BeforeTitle)
                Title=correct_line[BeforeTitle+8:AfterTitle-2]
                enddate=correct_line.find("-")
                Creationdate=correct_line[:enddate-1]
                Creator=correct_line[AfterTitle+12:]
                numParticipant=len(participant)
                Groupdata.update({"chat_name" : Title, "creation_date" : Creationdate, "num_of_participants" : numParticipant, "creator" : Creator})
    return Groupdata

In [8]:
def TotalMetadataOfGroup(fullmessages, Groupdata):
    Totalmetadata=dict()
    Totalmetadata.update({"messages" :  fullmessages})
    Totalmetadata.update({"metadata" :  Groupdata})
    return Totalmetadata


In [9]:
def FileToJson(Groupdata, Totalmetadata):
    import json
    with open(Groupdata["chat_name"]+ ".txt", 'w', encoding='utf-8') as ChatInJson_Textfile:
        json.dump(Totalmetadata, ChatInJson_Textfile,  ensure_ascii=False, indent=4)

In [13]:
file= getWhatsappTXT()
participant= NumOrNameToID(file)
fullmessages= DataOfMessages(file, participant)
Groupdata= DataOfGroup(file, participant)
Totalmetadata= TotalMetadataOfGroup(fullmessages, Groupdata)
FileToJson(Groupdata, Totalmetadata)